In [3]:
"""
learn.ipynb:
モデル学習用の一気通貫ファイル
Args:
    CheckFlag：1にすると処理、内部情報が閲覧用に出力されるモードになる。特に確認用と書かれているところはあとで消す．
"""
CheckFlag = 0


"""
各ライブラリのversionチェックを行うときに活用。
"""
from Cheack_Import_Library import Version_Check
Version_Check(CheckFlag)

In [4]:
"""
データセットのデータを読み込む。
Args:
    dataset_train : dataset/trainにあるimage,maskフォルダ内のファイルをそれぞれ訓練用のデータとして取り込む。
    
    dataset_val : dataset/validにあるimage,maskフォルダ内のファイルをそれぞれ交差検定用のデータとして取り込む。
    
"""
import os
from dataset import ShapesDataset

#訓練用のデータ
TRAIN_DATASET = os.path.join('dataset', 'train')
dataset_train = ShapesDataset()
dataset_train.load_dataset(TRAIN_DATASET)
dataset_train.prepare()

#交差検定用のデータ
VALID_DATASET = os.path.join('dataset', 'valid')
dataset_val = ShapesDataset()
dataset_val.load_dataset(VALID_DATASET)
dataset_val.prepare()

In [5]:
"""確認用、
取り込んだデータを画像として出力する。
CheckFlagが1ならば画像が表示される。    
"""
from mrcnn import visualize
image_ids = dataset_train.image_ids
for TRAIN_DATASET in image_ids:
    image = dataset_train.load_image(TRAIN_DATASET)
    mask, class_ids = dataset_train.load_mask(TRAIN_DATASET)
    if CheckFlag == 1:
        visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [6]:
"""確認用、
取り込んだ訓練データの配列構造が表示される。
CheckFlagが1ならば配列構造が表示される。    
"""
import pprint
if CheckFlag==1:
    pprint.pprint(vars(dataset_train))

In [7]:
"""
ハイパーパラメータと学習前のモデルをインポートする。 
CheckFlagが1ならばハイパーパラメータが表示される。 
"""
from config import ShapesConfig
import config
import mrcnn.model as modellib  

config = ShapesConfig()
if CheckFlag==1:
    config.display()

In [8]:
"""モデルを読み込む
どの初期モデルを用いるか？
時間がかかるのと，タスクマネージャーで専用GPUが消費されていることを確認できる．

最初だとmask_rcnn_coco.h5をインストール．
"""
from mrcnn import utils

model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir="logs/model")

COCO_MODEL_PATH = 'mask_rcnn_coco.h5'
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


In [10]:
"""GPU>1のときに用いる．
VRAMのリミッター解除
GPU_COUNT=1のときはすべてコメントアウトしてかまわない．
ここのconfigは上のハイパーパラメータとは異なるのだが，名前だけ書けるとうまく動かない汗

Args:
    config:TensorFlowのConfigであることに注意！　名前変えたいが，変えると動かぬ（直します）
"""

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()
config.allow_soft_placement=True
session = tf.Session(config=config)
KTF.set_session(session)

In [11]:
"""モデルの重みを読み込んでいるものと思われる．時間がかかる．
Args:


"""
model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])

In [12]:
"""モデルの訓練を行う．本学習．時間とGPUが消費される．
Args:
    learning_rate : 学習速度
    
    epochs : エポック数
    
    layers:

"""
model.train(dataset_train, dataset_val,
                learning_rate=0.01,
                epochs=1,
                layers='heads')


Starting at epoch 0. LR=0.01

Checkpoint Path: logs/model\cell_dataset20210409T1336\mask_rcnn_cell_dataset_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4 

C:\Users\トロコイド_山岡\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\トロコイド_山岡\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\トロコイド_山岡\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




Epoch 1/1
50/50 [==============================] - 150s 3s/step - loss: 1.4827 - rpn_class_loss: 0.0873 - rpn_bbox_loss: 0.5965 - mrcnn_class_loss: 0.0434 - mrcnn_bbox_loss: 0.4191 - mrcnn_mask_loss: 0.3363 - val_loss: 2.0375 - val_rpn_class_loss: 0.2624 - val_rpn_bbox_loss: 1.2490 - val_mrcnn_class_loss: 0.0358 - val_mrcnn_bbox_loss: 0.2595 - val_mrcnn_mask_loss: 0.2308
